<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Phoenix Prompts + OpenAI JavaScript SDK Tutorial</h1>

Let's see how to get started with using the Phoenix JavaScript SDK to pull prompts from an
instance of Phoenix, and then invoke those prompts using the OpenAI JavaScript SDK, all in a Deno Jupyter notebook.

> Note: that this example requires an OpenAI API key, and assumes you are running the Phoenix server on localhost:6006.

In [ ]:
import * as PhoenixClient from "npm:@arizeai/phoenix-client"
import OpenAI from "npm:openai"

Let's get started by creating a Phoenix client. This is technically optional, as the Phoenix Prompt helper methods will create a client if not provided.

However, we will create a client here to show how to configure the client with your Phoenix instance.

In [ ]:
const px = PhoenixClient.createClient({
  options: {
    baseUrl: "http://localhost:6006",
    // Uncomment this if you are using a Phoenix instance that requires an API key
    // headers: {
    //   Authorization: "bearer xxxxxx",
    // }
  }
})

Next, let's setup the OpenAI client. This will require an OpenAI API key setup in your environment.

In [ ]:
const apiKey = prompt("Enter your OpenAI API key:")

In [ ]:
const openai = new OpenAI({ apiKey: apiKey });

Now let's build a prompt in Phoenix!

We will create a prompt called `question-asker` that asks a question and then returns the answer.

Let's set up the prompt to use `OpenAI` as the LLM provider, and lets also add a "variable" to one of the prompt messages, named `question`.

This will allow us to pass in a question to the prompt when we invoke it

Once you have a prompt, you can pull it into the notebook using the Phoenix client.

In [ ]:
import * as Prompts from "npm:@arizeai/phoenix-client/prompts"
import { promptVersion } from "npm:@arizeai/phoenix-client/prompts"

const questionAskerPrompt = await Prompts.createPrompt({
 name: "question-asker",
 description: "Asks a question",
 version: promptVersion({
  description: "Initial version",
  templateFormat: "MUSTACHE",
  modelProvider: "OPENAI",
  modelName: "gpt-3.5-turbo",
  template: [
   {
     role: "user",
     content: "{{question}}"
   },
  ],
 })
})

await Deno.jupyter.md`
  ### question-asker prompt

  \`\`\`json
  ${JSON.stringify(questionAskerPrompt, null, 2)}
  \`\`\`
  `



In [ ]:
// We already have the prompt, but this just proves that we can fetch it from the server as well
const questionAskerPrompt = await Prompts.getPrompt({ client: px, prompt: { name: "question-asker" } })

await Deno.jupyter.md`
  ### question-asker prompt

  \`\`\`json
  ${JSON.stringify(questionAskerPrompt, null, 2)}
  \`\`\`
  `

Great! Now we have the contents of the `question-asker` prompt. It's currently in a format that Phoenix understands, so we need to convert it to a format that OpenAI can understand.

Luckily, `@arizeai/phoenix-client` has a helper function to convert the prompt to an OpenAI (or another LLM provider) format, and inject variables into the prompt at the same time. 

In [ ]:
const openaiPrompt = Prompts.toSDK({ 
  sdk: "openai", 
  prompt: questionAskerPrompt, 
  variables: { question: "How do I write Hello World in Deno?" } 
})

await Deno.jupyter.md`
  ### OpenAI Prompt

  \`\`\`json
  ${JSON.stringify(openaiPrompt, null, 2)}
  \`\`\`
`

We now have a set of "invocation parameters", fully typed and in the format that OpenAI expects.

Let's invoke OpenAI with our prompt and get our question answered!

In [ ]:
const response = await openai.chat.completions.create({
  ...openaiPrompt,
  // you can still override any of the invocation parameters as needed
  // for example, you can change the model or stream the response
  model: "gpt-4o-mini",
  stream: false
})

await Deno.jupyter.md`
  ### OpenAI Response

  ${response.choices[0].message.content}
`

Great! We've successfully invoked OpenAI with our prompt and got a response.

You have seen the basic steps to build a prompt in Phoenix, convert it to an OpenAI format, and invoke it.

Here are some additional features and capabilities you can explore:

- Prompt Versioning / Tagging
- Prompt Conversion to other LLM providers